<h1 align = "center">Review Scores Prediction on Olist</h1>

# Data Preparation

**Data Preparation Part** contains raw data cleaning, merging and precessing steps to get processed datasets we need;

Dataset from Kaggle : <i>"Brazilian E-Commerce Public Dataset by Olist"</i>  and  <i>"Marketing Funnel by Olist"</i>

Source:
https://www.kaggle.com/olistbr/brazilian-ecommerce and https://www.kaggle.com/olistbr/marketing-funnel-olist

Dara Itroduction:<br>
>1. A Brazilian ecommerce public dataset of orders made at Olist Store. The dataset has information of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil. Its features allows viewing an order from multiple dimensions: from order status, price, payment and freight performance to customer location, product attributes and finally reviews written by customers. We also released a geolocation dataset that relates Brazilian zip codes to lat/lng coordinates.<br>
<br>
>2. A marketing funnel dataset from sellers that filled-in requests of contact to sell their products on Olist Store. The dataset has information of 8k Marketing Qualified Leads (MQLs) that requested contact between Jun. 1st 2017 and Jun 1st 2018. They were randomly sampled from the total of MQLs.

---

#### Group Member (CZ115 FS6):
***Luo Yihang:*** &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*[U2022279G]*<br>
***Wnag Qianteng:*** *[U2022039K]*<br>
***Xue YuShan:*** &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*[U2022049H]*<br>

---


<a id="top"></a>
<h3 class="list-group-item list-group-item-action active" data-toggle="list" role="tab" aria-controls="home">Table of Content</h3>

* [1. Libraries](#1)
    <p></p>
* [2. Import Datasets](#2)
    - [Data Schema](#2.1) 
    <p></p>
* [3. Dataset 1 : Brazilian E-Commerce Public Dataset by Olist](#3) 
    - [3.1 Data Overview](#3.1)
    - [3.2 Data Merge](#3.2)
    - [3.3 Extract Relavant Columns](#3.3)
    - [3.4 Varibles Calculation](#3.4)
    <p></p>
* [4.  Dataset 2 : Marketing Funnel by Olist](#4)
    - [4.1 Data Overview](#4.1)
    - [4.2 Handle NULL Values](#4.2)
    - [4.3 Handle Data Types](#4.3)
    - [4.4 Add Review Scores of Sellers to the Dataset](#4.4)
    <p></p>
* [5. Processed Datasets Introduction](#5)

<a id="1"></a>
## 1. Libraries

In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import math
import os
sb.set() # set the default Seaborn style for graphics

<a id="2"></a>
## 2. Import Datasets

In [2]:
# Basic Dataset
print('### Brazilian E-Commerce Public Dataset by Olist ###')
for idx, file in enumerate(os.listdir('./brazilian-ecommerce')):
    print(idx, '-', file)
    
print('\n---------------------------------------------\n')

print('### Marketing Funnel by Olist ###')
for idx, file in enumerate(os.listdir('./marketing-funnel-olist')):
    print(idx, '-', file)

### Brazilian E-Commerce Public Dataset by Olist ###
0 - olist_customers_dataset.csv
1 - olist_geolocation_dataset.csv
2 - olist_orders_dataset.csv
3 - olist_order_items_dataset.csv
4 - olist_order_payments_dataset.csv
5 - olist_order_reviews_dataset.csv
6 - olist_products_dataset.csv
7 - olist_sellers_dataset.csv
8 - product_category_name_translation.csv
9 - review_sentiemnt.txt

---------------------------------------------

### Marketing Funnel by Olist ###
0 - data2cleaning.ipynb
1 - olist_closed_deals_dataset.csv
2 - olist_marketing_qualified_leads_dataset.csv
3 - problem1 .ipynb


In [3]:
# Reading all the files
raw_path = './brazilian-ecommerce/'
olist_customer = pd.read_csv(raw_path + 'olist_customers_dataset.csv')
olist_geolocation = pd.read_csv(raw_path + 'olist_geolocation_dataset.csv')
olist_orders = pd.read_csv(raw_path + 'olist_orders_dataset.csv')
olist_order_items = pd.read_csv(raw_path + 'olist_order_items_dataset.csv')
olist_order_payments = pd.read_csv(raw_path + 'olist_order_payments_dataset.csv')
olist_order_reviews = pd.read_csv(raw_path + 'olist_order_reviews_dataset.csv')
olist_products = pd.read_csv(raw_path + 'olist_products_dataset.csv')
olist_sellers = pd.read_csv(raw_path + 'olist_sellers_dataset.csv')
review_sentiment = pd.read_table(raw_path + "review_sentiemnt.txt", sep='\t', names = ['order_id', 'positive', 'negative'])

raw_path = './marketing-funnel-olist/'
mql = pd.read_csv(raw_path + 'olist_marketing_qualified_leads_dataset.csv')
cd = pd.read_csv(raw_path + 'olist_closed_deals_dataset.csv')

<a id="2.1"></a>
**Data Schema**

![ecommerce](./image/ecommerce.png) ![funnel](./image/funnel.png)

<a id="3"></a>
## 3. Dataset 1 : Brazilian E-Commerce Public Dataset by Olist

<a id="3.1"></a>
### 3.1 Data Overview

In [4]:
# Collecting all datasets
datasets = [olist_customer, olist_geolocation, olist_orders, olist_order_items, olist_order_payments,
            olist_order_reviews, olist_products, olist_sellers, review_sentiment]
names = ['olist_customer', 'olist_geolocation', 'olist_orders', 'olist_order_items', 'olist_order_payments',
         'olist_order_reviews', 'olist_products', 'olist_sellers', 'review_sentiemnt']

# Creating a DataFrame with useful information about all datasets
data_info = pd.DataFrame({})
data_info['dataset'] = names
data_info['n_rows'] = [df.shape[0] for df in datasets]
data_info['n_cols'] = [df.shape[1] for df in datasets]
data_info['null_amount'] = [df.isnull().sum().sum() for df in datasets]
data_info['qty_null_columns'] = [len([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]
data_info['null_columns'] = [', '.join([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]

data_info.style.background_gradient()

,dataset,n_rows,n_cols,null_amount,qty_null_columns,null_columns
0,olist_customer,99441,5,0,0,
1,olist_geolocation,1000163,5,0,0,
2,olist_orders,99441,8,4908,3,"order_approved_at, order_delivered_carrier_date, order_delivered_customer_date"
3,olist_order_items,112650,7,0,0,
4,olist_order_payments,103886,5,0,0,
5,olist_order_reviews,100000,7,146532,2,"review_comment_title, review_comment_message"
6,olist_products,32951,9,2448,8,"product_category_name, product_name_lenght, product_description_lenght, product_photos_qty, product_weight_g, product_length_cm, product_height_cm, product_width_cm"
7,olist_sellers,3095,4,0,0,
8,review_sentiemnt,100000,3,0,0,


<a id="3.2"></a>
### 3.2 Data Merge

In [5]:
# Merge all datasets into one dataframe according to data schema above
ecom = pd.merge(olist_orders,olist_order_items, on='order_id', how='inner')
ecom = pd.merge(ecom,olist_order_reviews, on='order_id', how='inner')
ecom = pd.merge(ecom,olist_order_payments, on='order_id', how='inner')
ecom = pd.merge(ecom,olist_customer, on='customer_id', how='inner')
ecom = pd.merge(ecom,olist_products, on='product_id', how='inner')
ecom = pd.merge(ecom,olist_sellers, on='seller_id', how='inner')
ecom = pd.merge(ecom,review_sentiment, on='order_id', how='inner')

olist_geolocation = olist_geolocation.drop_duplicates(['geolocation_zip_code_prefix'])   # Drop duplicates
olist_geolocation = olist_geolocation.drop(columns = ['geolocation_city','geolocation_state'])

olist_geolocation.columns = ['customer_zip_code_prefix', 'customer_lat', 'customer_lng']
ecom = pd.merge(ecom,olist_geolocation, on='customer_zip_code_prefix', how='inner')
olist_geolocation.columns = ['seller_zip_code_prefix', 'seller_lat', 'seller_lng']
ecom = pd.merge(ecom,olist_geolocation, on='seller_zip_code_prefix', how='inner')

In [6]:
# Only orders successfully delivered will be reserved
ecom = ecom[ecom['order_status'] == "delivered"]

# Drop NULL values from 'product_category_name' and 'product_weight_g'
ecom=ecom.dropna(subset=['product_category_name'])
ecom=ecom.dropna(subset=['product_weight_g'])

In [7]:
# Calculate the distance between sellers and customers
ecom['a'] = ecom['customer_lat'] - ecom['seller_lat']
ecom['b'] = ecom['customer_lng'] - ecom['seller_lng']
ecom['distance'] = ecom.apply(lambda ecom: 2*np.arcsin(math.sqrt(pow(math.sin(ecom['a'] /2*math.pi/180),2)+math.cos(ecom['customer_lat']*math.pi/180)*math.cos(ecom['seller_lat']*math.pi/180)*pow(math.sin(ecom['b'] /2*math.pi/180),2)))*6378.173,axis = 1)
ecom = ecom.drop(columns=['order_status','review_id','customer_unique_id','customer_zip_code_prefix','seller_zip_code_prefix','a','b'])  # Drop irrelevant columns

In [8]:
# Calculate the dilivery time and estimated delivery time from timestamps
# Extract columns into datetime
ecom['order_purchase_timestamp'] = pd.to_datetime(ecom['order_purchase_timestamp'])
ecom['order_approved_at'] = pd.to_datetime(ecom['order_approved_at'])
ecom['order_estimated_delivery_date'] = pd.to_datetime(ecom['order_estimated_delivery_date'])
ecom['order_delivered_customer_date'] = pd.to_datetime(ecom['order_delivered_customer_date'])

# Calculate delivery time in days
ecom['delivery_time'] = (ecom['order_delivered_customer_date'] - ecom['order_approved_at']).dt.total_seconds() / 86400
ecom['estimated_delivery_time'] = (ecom['order_estimated_delivery_date'] - ecom['order_approved_at']).dt.total_seconds() / 86400

In [9]:
# Information of the dataset
ecom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114892 entries, 0 to 119155
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       114892 non-null  object        
 1   customer_id                    114892 non-null  object        
 2   order_purchase_timestamp       114892 non-null  datetime64[ns]
 3   order_approved_at              114878 non-null  datetime64[ns]
 4   order_delivered_carrier_date   114890 non-null  object        
 5   order_delivered_customer_date  114884 non-null  datetime64[ns]
 6   order_estimated_delivery_date  114892 non-null  datetime64[ns]
 7   order_item_id                  114892 non-null  int64         
 8   product_id                     114892 non-null  object        
 9   seller_id                      114892 non-null  object        
 10  shipping_limit_date            114892 non-null  object        
 11  

<a id="3.3"></a>
### 3.3 Extract Relavant Columns
In our analysis, we don't need to use so many columns.

In [10]:
# Extract the variables needed
olist_ecommerce = pd.DataFrame(ecom[['product_id','order_id','customer_id',
                                     'delivery_time', 'estimated_delivery_time',
                                     'freight_value', 'distance','review_score','product_category_name',
                                     'product_description_lenght','product_photos_qty',
                                     'price', 'order_item_id','positive','negative']])

<a id="3.4"></a>
### 3.4 Varibles Calculation
Our goal is to predict the review score of products, so the varivles should be related to products instead of orders.

In [11]:
# Total sales
order_num = pd.DataFrame(olist_ecommerce.groupby('product_id')['order_item_id'].agg('sum'))
order_num.rename(columns = {'order_item_id': 'sales'}, inplace = True)
olist_ecommerce = pd.merge(olist_ecommerce, order_num, on="product_id", how="inner")

In [12]:
# Average of freight value
freight_value_ave = pd.DataFrame(olist_ecommerce.groupby('product_id')['freight_value'].agg('mean'))
freight_value_ave.rename(columns = {'freight_value': 'freight_value_ave'}, inplace = True)
olist_ecommerce = pd.merge(olist_ecommerce, freight_value_ave, on="product_id", how="inner")

In [13]:
# Average of delivery_time
delivery_time_ave = pd.DataFrame(olist_ecommerce.groupby('product_id')['delivery_time'].agg('mean'))
delivery_time_ave.rename(columns = {'delivery_time': 'delivery_time_ave'}, inplace = True)
olist_ecommerce = pd.merge(olist_ecommerce, delivery_time_ave, on="product_id", how="inner")

In [14]:
# Average of estimated delivery_time
estimated_delivery_time_ave = pd.DataFrame(olist_ecommerce.groupby('product_id')['estimated_delivery_time'].agg('mean'))
estimated_delivery_time_ave.rename(columns = {'estimated_delivery_time': 'estimated_delivery_time_ave'}, inplace = True)
olist_ecommerce = pd.merge(olist_ecommerce, estimated_delivery_time_ave, on="product_id", how="inner")

In [15]:
# Average of distance
distance_ave = pd.DataFrame(olist_ecommerce.groupby('product_id')['distance'].agg('mean'))
distance_ave.rename(columns = {'distance': 'distance_ave'}, inplace = True)
olist_ecommerce = pd.merge(olist_ecommerce, distance_ave, on="product_id", how="inner")

In [16]:
# Average of review score
review_score_ave = pd.DataFrame(olist_ecommerce.groupby('product_id')['review_score'].agg('mean'))
review_score_ave.rename(columns = {'review_score': 'review_score_ave'}, inplace = True)
olist_ecommerce = pd.merge(olist_ecommerce, review_score_ave, on="product_id", how="inner")

In [17]:
# Average of product_description_lenght
product_description_lenght_ave = pd.DataFrame(olist_ecommerce.groupby('product_id')['product_description_lenght'].agg('mean'))
product_description_lenght_ave.rename(columns = {'product_description_lenght': 'product_description_lenght_ave'}, inplace = True)
olist_ecommerce = pd.merge(olist_ecommerce, product_description_lenght_ave, on="product_id", how="inner")

In [18]:
# Average of product_photos_qty
product_photos_qty_ave = pd.DataFrame(olist_ecommerce.groupby('product_id')['product_photos_qty'].agg('mean'))
product_photos_qty_ave.rename(columns = {'product_photos_qty': 'product_photos_qty_ave'}, inplace = True)
olist_ecommerce = pd.merge(olist_ecommerce, product_photos_qty_ave, on="product_id", how="inner")

In [19]:
# Average of price
price_ave = pd.DataFrame(olist_ecommerce.groupby('product_id')['price'].agg('mean'))
price_ave.rename(columns = {'price': 'price_ave'}, inplace = True)
olist_ecommerce = pd.merge(olist_ecommerce, price_ave, on="product_id", how="inner")

In [20]:
# Average of positive sentiment score
positive_ave = pd.DataFrame(olist_ecommerce.groupby('product_id')['positive'].agg('mean'))
positive_ave.rename(columns = {'positive': 'positive_ave'}, inplace = True)
olist_ecommerce = pd.merge(olist_ecommerce, positive_ave, on="product_id", how="inner")

In [21]:
# Average of negative sentiment score
negative_ave = pd.DataFrame(olist_ecommerce.groupby('product_id')['negative'].agg('mean'))
negative_ave.rename(columns = {'negative': 'negative_ave'}, inplace = True)
olist_ecommerce = pd.merge(olist_ecommerce, negative_ave, on="product_id", how="inner")

In [22]:
# Now, we have varibles related to each product. Therefore, we can drop duplicate values of 'product_id'
olist_ecommerce = olist_ecommerce.drop_duplicates(['product_id'])

# Drop irrelevant columns
olist_ecommerce = olist_ecommerce.drop(columns=['order_id', 'customer_id', 'delivery_time','estimated_delivery_time', 
                                                'freight_value', 'distance', 'review_score',
                                                'product_description_lenght', 'product_photos_qty', 'price',
                                                'order_item_id', 'positive', 'negative',])

In [23]:
# The final dataset 1
olist_ecommerce.head()

,product_id,product_category_name,sales,freight_value_ave,delivery_time_ave,estimated_delivery_time_ave,distance_ave,review_score_ave,product_description_lenght_ave,product_photos_qty_ave,price_ave,positive_ave,negative_ave
0,87285b34884572647811a353c7ac498a,utilidades_domesticas,6,9.303333,7.785986,14.983281,94.932814,4.000000,268.0,4.0,29.990000,1.000000,-1.166667
6,b00a32a0b42fd65efb58a5822009f629,bebes,9,10.435556,6.722681,15.945769,166.680711,4.000000,398.0,3.0,77.677778,1.222222,-1.111111
15,725cbfcaff95a4d43742fdf13cf43c75,cool_stuff,24,11.344167,6.366090,16.799455,276.714411,4.416667,537.0,2.0,44.524583,1.208333,-1.000000
39,7e0dc102074f8285580c9777f79c90cf,ferramentas_jardim,79,13.710282,10.785287,26.305938,433.695308,3.774648,1251.0,2.0,35.956338,1.126761,-1.098592
110,a298a105818dce6878b787e4af6cff7d,bebes,38,11.974737,8.232452,18.539242,348.895663,4.710526,1158.0,2.0,79.900000,1.078947,-1.000000


<a id="4"></a>
## 4. Dataset 2 : Marketing Funnel by Olist

<a id="4.1"></a>
### 4.1 Data Overview

In [24]:
mf = mql.merge(cd, on='mql_id', how='left')

#Check the number of NaN values in each column
print('There are ',mf.shape[0],'rows and ',mf.shape[1],'columns in the merged dataset.')
print()
print(mf.isnull().sum())

closed_deals_count = cd['mql_id'].count()
total_mqls = mql['mql_id'].count()
closed_deal_rate = closed_deals_count/total_mqls.round(2)*100
print()

#Calculate the percentage of closed deals
print('Among the leads who had contacted to Olist,',closed_deal_rate,'% of their deals are closed at last.')

There are  8000 rows and  17 columns in the merged dataset.

mql_id                              0
first_contact_date                  0
landing_page_id                     0
origin                             60
seller_id                        7158
sdr_id                           7158
sr_id                            7158
won_date                         7158
business_segment                 7159
lead_type                        7164
lead_behaviour_profile           7335
has_company                      7937
has_gtin                         7936
average_stock                    7934
business_type                    7168
declared_product_catalog_size    7931
declared_monthly_revenue         7158
dtype: int64

Among the leads who had contacted to Olist, 10.525 % of their deals are closed at last.


<a id="4.2"></a>
### 4.2 Handle NULL Values

In [25]:
# We only analyze the closed deals
mf=mf.dropna(subset=['won_date'])
print('Missing value proportion(%):')
print((mf.isnull().sum()/mf.shape[0]*100).round(2))

Missing value proportion(%):
mql_id                            0.00
first_contact_date                0.00
landing_page_id                   0.00
origin                            1.66
seller_id                         0.00
sdr_id                            0.00
sr_id                             0.00
won_date                          0.00
business_segment                  0.12
lead_type                         0.71
lead_behaviour_profile           21.02
has_company                      92.52
has_gtin                         92.40
average_stock                    92.16
business_type                     1.19
declared_product_catalog_size    91.81
declared_monthly_revenue          0.00
dtype: float64


We can see variables 'has_company', 'has_gtin','average_stock', 'declared_product_catalog_size' have too many missing values, which means that they have poor value to analyze.

In [26]:
mf = mf.drop(columns=['has_company','has_gtin','average_stock','declared_product_catalog_size'])
mf = mf.drop(columns = ['sdr_id','sr_id','landing_page_id'])

<a id="4.3"></a>
### 4.3 Handle Data Types

In [27]:
#Convert columns of date and time to type datetime
mf['first_contact_date'] = pd.to_datetime(mf['first_contact_date'])
mf['won_date'] = pd.to_datetime(mf['won_date'])

#Calculate days from close
mf['days_to_close'] = (mf['won_date'] - mf['first_contact_date']).dt.days

In [28]:
#Convert categorical variables into category datatype
mf['origin'] = mf['origin'].astype('category')
mf['business_segment'] = mf['business_segment'].astype('category')
mf['lead_behaviour_profile'] = mf['lead_behaviour_profile'].astype('category')
mf['lead_type'] = mf['lead_type'].astype('category')
mf['business_type'] = mf['business_type'].astype('category')
mf.dtypes

mql_id                              object
first_contact_date          datetime64[ns]
origin                            category
seller_id                           object
won_date                    datetime64[ns]
business_segment                  category
lead_type                         category
lead_behaviour_profile            category
business_type                     category
declared_monthly_revenue           float64
days_to_close                        int64
dtype: object

<a id="4.4"></a>
### Add Review Scores of Sellers to the Dataset

In [29]:
# Calculate average review scores of each seller in the dataset
reviews_seller = pd.merge(olist_orders,olist_order_items, on='order_id', how='inner')
reviews_seller = pd.merge(reviews_seller,olist_order_reviews, on='order_id', how='inner')
reviews_seller = pd.merge(reviews_seller,olist_sellers, on='seller_id', how='inner')
review_score_ave = pd.DataFrame(reviews_seller.groupby('seller_id')['review_score'].agg('mean'))
review_score_ave.rename(columns = {'review_score': 'review_score_ave'}, inplace = True)
reviews_seller = pd.merge(reviews_seller, review_score_ave, on="seller_id", how="inner")
reviews_seller = reviews_seller[['seller_id','review_score_ave']]

# Merge the review scores dataset and sellers dataset
mf = pd.merge(mf,reviews_seller, on='seller_id', how='inner')

In [30]:
# The final dataset 2
mf.head()

,mql_id,first_contact_date,origin,seller_id,won_date,business_segment,lead_type,lead_behaviour_profile,business_type,declared_monthly_revenue,days_to_close,review_score_ave
0,5420aad7fec3549a85876ba1c529bd84,2018-02-21,organic_search,2c43fb513632d29b3b58df74816f1b06,2018-02-26 19:58:54,pet,online_medium,cat,reseller,0.0,5,3.666667
1,5420aad7fec3549a85876ba1c529bd84,2018-02-21,organic_search,2c43fb513632d29b3b58df74816f1b06,2018-02-26 19:58:54,pet,online_medium,cat,reseller,0.0,5,3.666667
2,5420aad7fec3549a85876ba1c529bd84,2018-02-21,organic_search,2c43fb513632d29b3b58df74816f1b06,2018-02-26 19:58:54,pet,online_medium,cat,reseller,0.0,5,3.666667
3,327174d3648a2d047e8940d7d15204ca,2018-04-03,organic_search,612170e34b97004b3ba37eae81836b4c,2018-06-05 17:27:23,home_appliances,online_big,cat,reseller,0.0,63,4.427273
4,327174d3648a2d047e8940d7d15204ca,2018-04-03,organic_search,612170e34b97004b3ba37eae81836b4c,2018-06-05 17:27:23,home_appliances,online_big,cat,reseller,0.0,63,4.427273


<a id="5"></a>
## 5. Processed Datasets Introduction

**ecom**: <br>The dataset includes all the inforamtion of each *order* on Olist in this period;<br>

**olist_ecommerce**: <br>The dataset includes the important inforamtion of each each *product* on Olist during this period;<br>

**mf**: <br>The dataset includes the marketing funnel information of some *sellers* on Olist during this period.

---

# Data Preparation Finished
<a href="#top"><button type="button">Back to Top</button></a><br><br>
<a href="data_analysis.html" target=_blank><button type="button">Data Analysis Part</button></a>